In [85]:
import os
import tensorflow as tf
import numpy as np
import pandas as pd
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization, Rescaling
from tensorflow.keras.callbacks import EarlyStopping

In [74]:
################### Here we are loading the flowers dataset, we will have to replace it with the one we need ###################
option_1 = True

raw_data_path = '../raw_data'

if option_1:
    !wget https://wagon-public-datasets.s3.amazonaws.com/flowers-dataset.zip -P {raw_data_path}
    !unzip flowers-dataset.zip -d {raw_data_path}
else:
    from google.colab import drive
    drive.mount('/content/drive/')

!ls {raw_data_path}

--2024-06-04 14:14:19--  https://wagon-public-datasets.s3.amazonaws.com/flowers-dataset.zip
Resolving wagon-public-datasets.s3.amazonaws.com (wagon-public-datasets.s3.amazonaws.com)... 52.218.108.210, 52.92.17.17, 52.218.24.130, ...
Connecting to wagon-public-datasets.s3.amazonaws.com (wagon-public-datasets.s3.amazonaws.com)|52.218.108.210|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 104983809 (100M) [application/zip]
Saving to: ‘../raw_data/flowers-dataset.zip’

flowers-dataset.zip 100%[===================>] 100.12M  4.80MB/s    in 32s     

2024-06-04 14:14:51 (3.13 MB/s) - ‘../raw_data/flowers-dataset.zip’ saved [104983809/104983809]

unzip:  cannot find or open flowers-dataset.zip, flowers-dataset.zip.zip or flowers-dataset.zip.ZIP.
flowers-dataset.zip


Use the following method to create 
`X_train, y_train, X_val, y_val, X_test, y_test, num_classes` depending on the `loading_method` you have used

In [59]:
from tensorflow.keras.utils import to_categorical
from tqdm import tqdm
import numpy as np
import os
from PIL import Image

def load_flowers_data(loading_method):
    if loading_method == 'colab':
        data_path = '/content/drive/My Drive/Deep_learning_data/flowers'
    elif loading_method == 'direct':
        data_path = 'flowers/'
    classes = {'daisy':0, 'dandelion':1, 'rose':2}
    imgs = []
    labels = []
    for (cl, i) in classes.items():
        images_path = [elt for elt in os.listdir(os.path.join(data_path, cl)) if elt.find('.jpg')>0]
        for img in tqdm(images_path[:300]):
            path = os.path.join(data_path, cl, img)
            if os.path.exists(path):
                image = Image.open(path)
                image = image.resize((256, 256))
                imgs.append(np.array(image))
                labels.append(i)

    X = np.array(imgs)
    num_classes = len(set(labels))
    y = to_categorical(labels, num_classes)
     # Finally we shuffle:
    p = np.random.permutation(len(X))
    X, y = X[p], y[p]

    first_split = int(len(imgs) /6.)
    second_split = first_split + int(len(imgs) * 0.2)
    X_test, X_val, X_train = X[:first_split], X[first_split:second_split], X[second_split:]
    y_test, y_val, y_train = y[:first_split], y[first_split:second_split], y[second_split:]

    return X_train, y_train, X_val, y_val, X_test, y_test, num_classes


In [60]:
# CALL load_flowers_data WITH YOUR PREFERRED METHOD HERE
# $DELETE_BEGIN
X_train, y_train, X_val, y_val, X_test, y_test, num_classes = load_flowers_data('direct')
# $DELETE_END

100%|██████████| 299/299 [00:03<00:00, 91.22it/s] 


In [61]:
print(X_train.shape, X_val.shape, X_test.shape)

(571, 256, 256, 3) (179, 256, 256, 3) (149, 256, 256, 3)


In [62]:
y_train

array([[0., 1., 0.],
       [0., 1., 0.],
       [1., 0., 0.],
       ...,
       [0., 0., 1.],
       [0., 1., 0.],
       [1., 0., 0.]], dtype=float32)

In [63]:
# Define image parameters
IMG_HEIGHT = 256 #512
IMG_WIDTH = 256 #512
BATCH_SIZE = 32
NUM_CLASSES = 15

In [64]:
# Define a simple model
def simple_model():

    model = Sequential()
    model.add(Rescaling(1./255, input_shape=(IMG_HEIGHT,IMG_WIDTH,3)))

    model.add(Conv2D(16, kernel_size=10, activation='relu'))
    model.add(MaxPooling2D(3))

    model.add(Conv2D(32, kernel_size=8, activation="relu"))
    model.add(MaxPooling2D(3))

    model.add(Conv2D(32, kernel_size=6, activation="relu"))
    model.add(MaxPooling2D(3))

    model.add(Flatten())
    model.add(Dense(100, activation='relu'))
    model.add(Dense(3, activation='softmax'))

    model.compile(loss='categorical_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])

    return model


In [65]:
s_model = simple_model()

In [66]:
es = EarlyStopping(monitor='val_loss', patience=5, verbose=0, restore_best_weights=True)

In [67]:
s_model.summary()

Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 rescaling_7 (Rescaling)     (None, 256, 256, 3)       0         
                                                                 
 conv2d_15 (Conv2D)          (None, 247, 247, 16)      4816      
                                                                 
 max_pooling2d_15 (MaxPooli  (None, 82, 82, 16)        0         
 ng2D)                                                           
                                                                 
 conv2d_16 (Conv2D)          (None, 75, 75, 32)        32800     
                                                                 
 max_pooling2d_16 (MaxPooli  (None, 25, 25, 32)        0         
 ng2D)                                                           
                                                                 
 conv2d_17 (Conv2D)          (None, 20, 20, 32)       

In [68]:
# Train the model
history = s_model.fit(
    X_train,
    y_train,
    validation_data=(X_val, y_val),
    epochs=10,
    batch_size=BATCH_SIZE,
    callbacks=[es])

Epoch 1/10
18/18 [==============================] - 9s 431ms/step - loss: 1.0133 - accuracy: 0.4378 - val_loss: 0.8316 - val_accuracy: 0.5419
Epoch 2/10
18/18 [==============================] - 8s 441ms/step - loss: 0.9202 - accuracy: 0.5236 - val_loss: 0.8488 - val_accuracy: 0.5307
Epoch 3/10
18/18 [==============================] - 7s 406ms/step - loss: 0.8329 - accuracy: 0.5884 - val_loss: 0.8387 - val_accuracy: 0.5307
Epoch 4/10
18/18 [==============================] - 7s 412ms/step - loss: 0.7463 - accuracy: 0.6760 - val_loss: 0.6797 - val_accuracy: 0.7039
Epoch 5/10
18/18 [==============================] - 7s 419ms/step - loss: 0.6734 - accuracy: 0.7461 - val_loss: 0.7194 - val_accuracy: 0.6983
Epoch 6/10
18/18 [==============================] - 7s 415ms/step - loss: 0.5933 - accuracy: 0.7566 - val_loss: 0.7512 - val_accuracy: 0.6648
Epoch 7/10
18/18 [==============================] - 8s 420ms/step - loss: 0.5499 - accuracy: 0.7898 - val_loss: 0.6596 - val_accuracy: 0.7263
Epoch 

In [80]:
test_loss, test_accuracy = s_model.evaluate(X_test, y_test)
val_loss, val_accuracy = s_model.evaluate(X_val, y_val)

6/6 [==============================] - 1s 150ms/step - loss: 0.8966 - accuracy: 0.6983


In [81]:
print('val accuracy:', val_accuracy)
print('test accuracy:', test_accuracy)

val accuracy: 0.6983240246772766
test accuracy: 0.7651006579399109


In [87]:
from tensorflow.keras.models import save_model

models_dir = '../models'

# Ensure that the models directory exists
os.makedirs(models_dir, exist_ok=True)


# Save the trained model to the models directory
save_model(s_model, os.path.join(models_dir, 'simple_model.h5'))

/tmp/ipykernel_15487/4256656333.py:6: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  save_model(s_model, os.path.join(models_dir, 'simple_model.h5'))
